# 集成以往写出来的一些小函数并在一个小数据集上测试
# write some functions and test them on a small dataset

环境设置 set the environments

In [25]:
import math
import chess
import os
import sys
import chess.pgn
import chess.engine
import csv
import math
from tqdm import tqdm
import numpy as np
import pandas as pd
import asyncio
from tqdm import tqdm
# 在代码头部添加（重要！）
from stockfish import Stockfish  # 确保此语句在调用Stockfish之前
# 🔧 关键：强制更换 event loop（解决部分 Windows 异步问题）
if sys.platform.startswith('win'):
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())


一些文件路径、分析深度等初始设置

In [26]:
csv_file="C:\\Users\\Administrator\\Desktop\\simple eda\\testfunctions\\original_dataset.csv"
STOCKFISH_PATH ="C:/Users/Administrator/Desktop/stockfish-windows-x86-64-avx2/stockfish/stockfish-windows-x86-64-avx2.exe"
STOCKFISH_DEPTH=20
OUTPUT_CSV = "test_check.csv"  # 输出路径

In [27]:
# 计算胜率差（和原公式一致）
def _calculate_delta(cp_before, cp_after):
    def cp_to_win(cp):
        return 50 + 50 * (2 / (1 + math.exp(-0.00368208 * cp)) - 1)
    return abs(cp_to_win(cp_before) - cp_to_win(cp_after))
# 辅助函数：从 Stockfish 的评估字典中提取 cp 分值
def get_cp(evaluation):
    # evaluation 形如 {'type': 'cp', 'value': 34} 或 {'type': 'mate', 'value': 2}
    if evaluation["type"] == "cp":
        return evaluation["value"]
    elif evaluation["type"] == "mate":
        # 如果是 mate，按正负10000 处理
        return 10000 if evaluation["value"] > 0 else -10000
    return 0

测试上面的能不能用

In [28]:
# 逐步分析棋局准确性（完全按照你原来的评估逻辑）
def generate_mediate_df(csv_file, STOCKFISH_PATH, depth=20):
    raw_df = pd.read_csv(csv_file, encoding='latin1')
    mediate_df = raw_df.copy()
    mediate_df['white_accuracy'] = None
    mediate_df['black_accuracy'] = None

    # 使用 stockfish 包来替代 chess.engine.SimpleEngine
    from stockfish import Stockfish
    stockfish = Stockfish(path=STOCKFISH_PATH, depth=depth)
    # 若需要，可设置其他参数，例如线程数、hash 大小等

    with tqdm(total=len(mediate_df),
              desc="🔄 棋局分析进度",
              unit="局",
              bar_format="{l_bar}{bar:30}| {n_fmt}/{total_fmt} [剩余:{remaining}]",
              dynamic_ncols=True) as pbar:

        for idx, row in mediate_df.iterrows():
            pbar.set_postfix_str(f"当前对局ID: {idx}")
            moves = str(row['Moves']).split(',')
            board = chess.Board()
            white_acc = []
            black_acc = []

            for move_uci in moves:
                move_uci = move_uci.strip()
                if not move_uci:
                    continue

                try:
                    move = chess.Move.from_uci(move_uci)
                    if move not in board.legal_moves:
                        # 如果遇到非法走法则跳出本局分析
                        break

                    fen_before=board.fen()
                    stockfish.set_fen_position(fen_before)
                    eval_before=stockfish.get_evaluation()
                    cp_before=get_cp(eval_before)

                    board.push(move)

                    fen_after=board.fen()
                    stockfish.set_fen_position(fen_after)
                    eval_after=stockfish.get_evaluation()
                    cp_after=get_cp(eval_after)
                


                    # 判断本步走法是哪方走的：
                    # 注意：执行 push() 后，board.turn 表示下一步走棋的颜色，
                    # 所以走完走法前的局面时，当前走棋方为与 board.turn 相反的一方
                    moved_color = 'white' if board.turn == chess.BLACK else 'black'

                    # 根据原公式计算走法准确率
                    delta = _calculate_delta(cp_before, cp_after)
                    acc = 103.1668 * math.exp(-0.04354 * delta) - 3.1669
                    clamped_acc = max(0.0, min(100.0, acc))

                    if moved_color == 'white':
                        white_acc.append(clamped_acc)
                    else:
                        black_acc.append(clamped_acc)

                except Exception as e:
                    print(f"\n❌ 对局 {idx} 发生错误：{str(e)}")
                    break

            # 记录本局的平均准确率
            mediate_df.at[idx, 'white_accuracy'] = sum(white_acc) / len(white_acc) if white_acc else None
            mediate_df.at[idx, 'black_accuracy'] = sum(black_acc) / len(black_acc) if black_acc else None
            pbar.update(1)

    return mediate_df

In [29]:
# ===== 主函数入口 =====
if __name__ == "__main__":
    mediate_df = generate_mediate_df(
        csv_file=csv_file,
        STOCKFISH_PATH=STOCKFISH_PATH,
        depth=STOCKFISH_DEPTH
    )
    mediate_df.to_csv(OUTPUT_CSV, index=False)
    print(f"\n✅ 分析完成！结果已保存至: {OUTPUT_CSV}")

🔄 棋局分析进度:  46%|█████████████▊                | 6/13 [剩余:08:06]


❌ 对局 5 发生错误：expected uci string to be of length 4 or 5: 'nan'


🔄 棋局分析进度: 100%|██████████████████████████████| 13/13 [剩余:00:00]


✅ 分析完成！结果已保存至: test_check.csv


In [37]:
from stockfish import Stockfish

# 配置参数
STOCKFISH_DEPTH = 10
MISSED_WIN_THRESHOLD = 1000
THREADS = 4
HASH_SIZE = 2048  # MB

def get_score_diff(stockfish, fen, move_uci):
    """获取指定走法与最佳走法的评分差异"""
    stockfish.set_fen_position(fen)
    # 获取最佳走法评估
    best_move = stockfish.get_best_move(fen)
    if not best_move:
        return 0
    
    # 获取最佳走法评分
    best_eval = stockfish.get_evaluation()
    best_score = best_eval['value'] if best_eval['type'] == 'cp' else 10000 * (1 if best_eval['value'] > 0 else -1)    # 获取实际走法评分
    #实际走法评分
    stockfish.set_fen_position(fen)
    stockfish.make_moves_from_current_position([move_uci])
    actual_eval = stockfish.get_evaluation()
    actual_score = actual_eval['value'] if actual_eval['type'] == 'cp' else 10000* (1 if actual_eval['value'] > 0 else -1)
    
    # 计算差异（从行棋方角度）
    if stockfish.get_fen_position().split()[1] == 'b':  # 如果是黑方行棋
        return best_score - actual_score
    else:
        return actual_score - best_score
    
def calculate_acpl(moves_str, stockfish):
    """基于stockfish库的ACPL计算"""
    board = chess.Board()
    white_loss = white_moves = 0
    black_loss = black_moves = 0
    
    try:
        # 优化：根据UCI格式处理走法
        if ',' in moves_str:
            moves = [m.strip() for m in moves_str.split(',') if m.strip()]
        else:
            # 假设是空格分隔的UCI格式
            moves = [m.strip() for m in moves_str.split() if m.strip()]
            
        for move_uci in moves:
            if board.is_game_over():
                break
            
            try:
                move = chess.Move.from_uci(move_uci)
                if move not in board.legal_moves:
                    continue
            except:
                continue
            
            # 获取当前局面FEN和行棋方
            current_fen = board.fen()
            current_color = 'white' if board.turn == chess.WHITE else 'black'
            
            # 计算评分差异
            score_diff = get_score_diff(stockfish, current_fen, move_uci)
            score_diff=abs(score_diff)
            if score_diff >= MISSED_WIN_THRESHOLD:
                score_diff = 0
                
            # 累计损失
            if current_color == 'white':
                white_loss += score_diff
                white_moves += 1
            else:
                black_loss += score_diff
                black_moves += 1
                
            # 执行走子
            board.push(move)
            
    except Exception as e:
        print(f"处理错误: {str(e)}")
    
    # 计算平均值
    white_acpl = white_loss / white_moves if white_moves > 0 else 0
    black_acpl = black_loss / black_moves if black_moves > 0 else 0
    return round(white_acpl, 1), round(black_acpl, 1)

if __name__ == "__main__":
    # 初始化Stockfish引擎
    stockfish = Stockfish(
        path=STOCKFISH_PATH,
        depth=STOCKFISH_DEPTH,
        parameters={
            "Threads": THREADS,
            "Hash": HASH_SIZE,
            "UCI_Elo": 3200  # 可选：设置引擎强度
        }
    )
    
   
    
    # 处理数据
    white_acpl_list = []
    black_acpl_list = []

    # 添加批处理和进度显示
    batch_size = 10  # 每批处理的棋局数
    total_batches = (len(mediate_df) + batch_size - 1) // batch_size

    with tqdm(total=len(mediate_df), desc="ACPL计算进度", unit="局") as pbar:
        for i in range(0, len(mediate_df), batch_size):
            batch = mediate_df.iloc[i:i+batch_size]
            
            for idx, row in batch.iterrows():
                moves_str = row['Moves'] if isinstance(row['Moves'], str) else ''
                w_acpl, b_acpl = calculate_acpl(moves_str, stockfish)
                
                white_acpl_list.append(w_acpl)
                black_acpl_list.append(b_acpl)
                
                pbar.update(1)
                pbar.set_postfix_str(f"白ACPL: {w_acpl} | 黑ACPL: {b_acpl}")
    
    # 保存结果
    mediate_df["White_ACPL"] = white_acpl_list
    mediate_df["Black_ACPL"] = black_acpl_list
    mediate_df.to_csv("analyzed_games.csv", index=False)
    print(mediate_df.head())
    print(mediate_df.columns.tolist())


ACPL计算进度:   0%|          | 0/13 [10:55<?, ?局/s]


KeyboardInterrupt: 

求对局数量

In [38]:
#求moves

# 计算每行的移动步数并添加新列
mediate_df['Move Numbers'] = mediate_df['Moves'].apply(
    lambda x: x.count(',') + 1 if isinstance(x, str) and ',' in x and x.strip() else None
)
print(mediate_df.head())
print(mediate_df.columns.tolist())

                                    Event       Site        Date  Round  \
0  Early-Titled-Tuesday-Blitz-May-07-2024  Chess.com  2024.05.07      1   
1  Early-Titled-Tuesday-Blitz-May-07-2024  Chess.com  2024.05.07      1   
2  Early-Titled-Tuesday-Blitz-May-07-2024  Chess.com  2024.05.07      1   
3  Early-Titled-Tuesday-Blitz-May-07-2024  Chess.com  2024.05.07      1   
4  Early-Titled-Tuesday-Blitz-May-07-2024  Chess.com  2024.05.07      1   

         White             Black Result  WhiteElo  BlackElo TimeControl  \
0       Honiex            Senn71    1-0      2581      2201       180+1   
1    Aisen1011         GMSrinath    0-1      2475      2847       180+1   
2     Airquake       Road2GM3000    1-0      2824      2466       180+1   
3  miragha_yev         Eliwood44    1-0      2815      2455       180+1   
4      klimkoj  Happy1712Drummer    1-0      2921      2507       180+1   

       EndTime                     Termination  \
0  8:02:34 PDT       Honiex won by resignation  

In [39]:
print(mediate_df['white_accuracy'].head())
print(mediate_df['black_accuracy'].head())
print(mediate_df['Move Numbers'].head())


0    96.507881
1    91.925971
2    95.902978
3    93.777876
4    95.362976
Name: white_accuracy, dtype: object
0    83.715154
1    96.932798
2    87.022344
3    89.251709
4    88.042223
Name: black_accuracy, dtype: object
0    21.0
1    36.0
2    33.0
3    47.0
4    41.0
Name: Move Numbers, dtype: float64
